In [5]:
import mysql.connector
from mysql.connector.constants import ClientFlag

In [6]:
config = {
    'user': 'root',
    'password': 'Hh12H8y2ppBAq5Cj',
    'host': '34.65.175.109',
    'client_flags': [ClientFlag.SSL],
    'ssl_ca': '/Users/xavier/Desktop/EPFL_Cours/Ici/DB/db-54/part2/ssl/server-ca.pem',
    'ssl_cert': '/Users/xavier/Desktop/EPFL_Cours/Ici/DB/db-54/part2/ssl/client-cert.pem',
    'ssl_key': '/Users/xavier/Desktop/EPFL_Cours/Ici/DB/db-54/part2/ssl/client-key.pem',
    'database': 'data'
}

cnx = mysql.connector.connect(**config)
cursor = cnx.cursor()

# Party

In [11]:
cursor.execute("DROP TABLE OtherAssociatedFactors")
cursor.execute("CREATE TABLE OtherAssociatedFactors (id INTEGER,"
    "description CHAR(1) NOT NULL UNIQUE,"
    "CHECK (description BETWEEN 'A' AND 'Z'),"
    "PRIMARY KEY(id))")

In [12]:
cursor.execute("DROP TABLE CellphoneUse")
cursor.execute("""CREATE TABLE CellphoneUse(id INTEGER,
			   description CHAR(1) NOT NULL UNIQUE,
                CHECK (description BETWEEN 'B' AND 'D'),	
			   PRIMARY KEY(id))""")
# messy data: 1 -> B, 2 -> C, 3 -> D

In [14]:
cursor.execute("""CREATE TABLE FinancialResponsability(id INTEGER,
			   	      description CHAR(1) NOT NULL UNIQUE,
                                      CHECK (description IN ('N', 'Y', 'O', 'E', '-')),	
			              PRIMARY KEY(id))""")

# clean data 

In [16]:
cursor.execute("""CREATE TABLE MovementPrecedingCollision(id INTEGER,
					 description VARCHAR(100) NOT NULL UNIQUE,
					 PRIMARY KEY(id))""")
# varchar -> messy data, whatever.

In [17]:
cursor.execute("""CREATE TABLE PartyDrugPhysical(id INTEGER,
			       description CHAR(1) NOT NULL UNIQUE,
                            CHECK (description IN ('E', 'F', 'H', 'I')),
			       PRIMARY KEY(id))""")

# messy: letter G most prevalent: interpret as n/a ?

In [18]:
cursor.execute("""CREATE TABLE PartySafetyEquipement(id INTEGER,
				   description CHAR(1) NOT NULL UNIQUE,
                                   CHECK (description BETWEEN 'A' AND 'Y'),
				   PRIMARY KEY(id))""")
# clean as far as i can tell

The following table holds for `VictimSex` and `PartySex`

In [37]:
cursor.execute("""CREATE TABLE PersonSex(id INTEGER,
		       description VARCHAR(6) NOT NULL UNIQUE,
                       CHECK (description in ('male', 'female')),
		       PRIMARY KEY(id))""")
# clean af, for PartySex and VictimSex

In [20]:
cursor.execute("""CREATE TABLE PartySobriety(id INTEGER,
			    description CHAR(1) NOT NULL UNIQUE,
                            CHECK (description IN ('A', 'B', 'C', 'D', 'G', 'H')),
			    PRIMARY KEY(id))""")
# clean af

In [22]:
cursor.execute("""CREATE TABLE PartyType(id INTEGER,
			description VARCHAR(15) NOT NULL UNIQUE,
			PRIMARY KEY(id))""")

# clean

In [23]:
cursor.execute("""CREATE TABLE StatewideVehiculeType(id INTEGER,
				    description VARCHAR(30) NOT NULL UNIQUE,
				    PRIMARY KEY(id))""")
# clean as far as i can tell

In [24]:
cursor.execute("""CREATE TABLE VehiculeMake(id INTEGER,
			   description VARCHAR(100) NOT NULL,
			   PRIMARY KEY(id))""")

# clean

# Collision

In [25]:
cursor.execute("""CREATE TABLE CollisionSeverity(id INTEGER,
			       description VARCHAR(25) NOT NULL UNIQUE,
			       PRIMARY KEY(id))""")
# clean

In [26]:
cursor.execute("""CREATE TABLE HitAndRun(id INTEGER,
			description VARCHAR(20) NOT NULL UNIQUE,
			PRIMARY KEY(id))""")
# One dirty value: D -> NaN 

In [27]:
cursor.execute("""CREATE TABLE Lighting(id INTEGER,
		       description VARCHAR(35) NOT NULL UNIQUE,
		       PRIMARY KEY(id))""")
# Clean

In [28]:
cursor.execute("""CREATE TABLE LocationType(id INTEGER,
			   description VARCHAR(20) NOT NULL UNIQUE,
			   PRIMARY KEY(id))""")
# clean

In [29]:
cursor.execute("""CREATE TABLE PcfViolationCategory(id INTEGER,
				   description VARCHAR(70) NOT NULL UNIQUE,
				   PRIMARY KEY(id))""")

# One dirty value: 21804 -> NaN

In [30]:
cursor.execute("""CREATE TABLE PrimaryCollisionFactor(id INTEGER,
				     description VARCHAR(25) NOT NULL UNIQUE,
				     PRIMARY KEY(id))""")
# clean

In [31]:
cursor.execute("""CREATE TABLE RoadCondition(id INTEGER,
			    description VARCHAR(20) NOT NULL UNIQUE,
			    PRIMARY KEY(id))""")
# clean

In [32]:
cursor.execute("""CREATE TABLE RoadSurface(id INTEGER,
			  description VARCHAR(15) NOT NULL UNIQUE,
			  PRIMARY KEY(id))""")
# one dirty value: H -> NaN

In [33]:
cursor.execute("""CREATE TABLE TypeOfCollision(id INTEGER,
			      description VARCHAR(15) NOT NULL UNIQUE,
			      PRIMARY KEY(id))""")
# clean

In [34]:
cursor.execute("""CREATE TABLE Weather(id INTEGER,
		      description VARCHAR(10) NOT NULL UNIQUE,
		      PRIMARY KEY(id))""")
# clean

# Victim

In [36]:
cursor.execute("""CREATE TABLE VictimDegreeOfInjury(id INTEGER,
				   description VARCHAR(30) NOT NULL UNIQUE,
				   PRIMARY KEY(id))""")
# one drity value: 7 -> NaN